## Cards

In [19]:
import os
import json
import uuid
import pandas as pd
import datetime
import shutil
import csv

lookup_path = './lookup/'
source_path = './out/'
mappings_path = './mappings/'
#save_path = '../../../../digipolis-arches-shoku-pkg/source/'
source_file_name = './out/monuments.json'

## Functions

In [20]:
def resource_identifier(source_id):
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, source_id))


def generate_uri(source_id):
    return str(f'https://www.geonames.org/{source_id}')


def coordinates_geo_json(source_value): 
    
    return f'POINT {source_value}'.replace(",", "")


def coordinates_wkt(source_value):    
    return f'{source_value}'.replace(",", "")


def get_constants(node_mapping):
    constant_dict = {}
    constants = node_mapping['constants']
    for constant in constants:
        for constant_label, value in constant.items():
            constant_dict[constant_label] = value
    return constant_dict


def lookup(code, lookup_file):
    lookup_df = pd.read_csv(lookup_path + lookup_file)
    try:
        concept_name = lookup_df[lookup_df['code'] == code]['concept'].iloc[0]
        return concept_name
    except:
        print(f'{code} NOT FOUND IN {lookup_file}')
        return f'{code} NOT FOUND IN {lookup_file}'


## Save csv

In [21]:
def save_mapped_set(source_df, _package, _resource_model, _card):
    
    package = _package.split('_')[1]     
    
    file_name = '%s/%s/%s/%s.csv' % (save_path, package, _resource_model, _card)
    source_df.to_csv(file_name, index=False)    
    del source_df
    return 'Saved OK'

In [22]:
def related_resource(source_value):
    resourceid = str(uuid.uuid5(uuid.NAMESPACE_DNS, source_value))
    resourceXresourceId = str(uuid.uuid5(uuid.NAMESPACE_DNS, source_value + 'relation'))
    relation = '[{"resourceId": "%s", "ontologyProperty": "", "resourceXresourceId": "%s", "inverseOntologyProperty": ""}]' % (resourceid, resourceXresourceId)
    return relation

## Architects

In [23]:
def make_achitects(source_list, node_mappings, graph, dataset, card_name, card_order):
    
    obj_dict = {}
    obj_list = []
    
    for obj in source_list:
        
        if 'architectURI' in obj.keys():
        

            if isinstance(obj['architectURI'], dict):
                
                obj_dict['ResourceID'] = obj['ResourceID']
                obj_dict['card'] = 'architects'
                obj_dict['card_order'] = '5'
                architect_uri = str(uuid.uuid5(uuid.NAMESPACE_DNS, obj['architectURI']['Agent']['id']))
                
                resourceXresourceId = str(uuid.uuid5(uuid.NAMESPACE_DNS, obj['architectURI']['Agent']['id'] + 'relation'))                
                obj_dict['production_event_actor'] = '[{"resourceId": "%s", "ontologyProperty": "", "resourceXresourceId": "%s", "inverseOntologyProperty": ""}]' % (architect_uri, resourceXresourceId)
                obj_dict['production_event_actor_role'] = 'Architect'
                
                # for actors
                obj_dict['resource_id'] = architect_uri
                obj_dict['identifier_content'] = obj['architectURI']['Agent']['id']
                obj_dict['identifier_type'] = "Brabant Cloud ARK Identifier"
                obj_dict['architect_name'] = obj['architectURI']['Agent']['prefLabel']
                #obj_dict['production_event_timespan_start'] = obj['dateCreated']

                try:
                    obj_dict['alt_names'] = obj['architectURI']['Agent']['altLabel']
                except:
                    pass
                try:
                    obj_dict['other_identifier_content'] = obj['architectURI']['Agent']['wikidata']
                    obj_dict['other_identifier_type'] = 'Wikidata'
                except:
                    pass

                obj_list.append(obj_dict)
                obj_dict = {}

            elif isinstance(obj['architectURI'], list):   
                
                for architect in obj['architectURI']:                
                
                    obj_dict['ResourceID'] = obj['ResourceID']
                    obj_dict['card'] = 'architects'
                    obj_dict['card_order'] = '5'                    
                    architect_uri = str(uuid.uuid5(uuid.NAMESPACE_DNS, architect['Agent']['id']))
                    
                    resourceXresourceId = str(uuid.uuid5(uuid.NAMESPACE_DNS, architect['Agent']['id'] + 'relation'))                
                    obj_dict['production_event_actor'] = '[{"resourceId": "%s", "ontologyProperty": "", "resourceXresourceId": "%s", "inverseOntologyProperty": ""}]' % (architect_uri, resourceXresourceId)
                    obj_dict['production_event_actor_role'] = 'Architect'
                    #obj_dict['production_event_timespan_start'] = obj['dateCreated']

                    
                    # for actors
                    obj_dict['resource_id'] = architect_uri
                    obj_dict['identifier_content'] = architect['Agent']['id']
                    obj_dict['identifier_type'] = "Brabant Cloud ARK Identifier"
                    obj_dict['architect_name'] = architect['Agent']['prefLabel']
                    try:
                        obj_dict['alt_names'] = architect['Agent']['altLabel']
                    except:
                        pass
                    try:
                        obj_dict['other_identifier_content'] = architect['Agent']['wikidata']
                        obj_dict['other_identifier_type'] = 'Wikidata'
                    except:
                        pass
                    obj_list.append(obj_dict)
                    obj_dict = {}
            
            #resourceXresourceId = str(uuid.uuid5(uuid.NAMESPACE_DNS, obj_dict['material_location_street_address_city'] + 'relation'))
            #obj_dict['material_location_place'] = 
        
        
    a_source_df = pd.DataFrame(obj_list)
    number_of_records = len(obj_list)
        
    if number_of_records > 0:                       
        #save_it = save_mapped_set(source_df, package, resource_model, card)
        a_source_df.to_csv(f'out/buildings/architects.csv', index =False)
        #return '%s saved: %s' % (card_name, number_of_records)
        return a_source_df


#relation = '[{"resourceId": "%s", "ontologyProperty": "", "resourceXresourceId": "%s", "inverseOntologyProperty": ""}]' % (usr_dict['ResourceID_mat'] , uuid.uuid4())


## Dedications

In [24]:
def make_dedication_actor(obj_dict):

    act_dict = {}
    act_dict['name'] = obj['dedicationURI']['Concept']['prefLabel']
    act_dict['alt_names'] = obj['dedicationURI']['Concept']['altLabel']
    act_dict['wikidata'] = obj['dedicationURI']['Concept']['wikidata']
    act_dict['wikidata_type'] = 'Wikidata'

def make_dedications(source_list, node_mappings, graph, dataset, card_name, card_order):
    
    obj_dict = {}
    obj_list = []
    
    for obj in source_list:
        
        if 'dedicationURI' in obj.keys():
        

            if isinstance(obj['dedicationURI'], dict):
                
                obj_dict['ResourceID'] = obj['ResourceID']
                obj_dict['card'] = "dedications"
                obj_dict['card_order'] = '6'
                dedication_uri = str(uuid.uuid5(uuid.NAMESPACE_DNS, obj['dedicationURI']['Concept']['id']))
                resourceXresourceId = str(uuid.uuid5(uuid.NAMESPACE_DNS, obj['dedicationURI']['Concept']['id'] + 'relation'))                
                obj_dict['dedication_activity_actor'] = '[{"resourceId": "%s", "ontologyProperty": "", "resourceXresourceId": "%s", "inverseOntologyProperty": ""}]' % (dedication_uri, resourceXresourceId)
                obj_dict['dedication_activity_actor_role'] = 'Dedicated to'                
                obj_dict['resource_id'] = dedication_uri
                # for actors
                obj_dict['resource_id'] = dedication_uri
                obj_dict['identifier_content'] = obj['dedicationURI']['Concept']['id']
                obj_dict['identifier_type'] = "Brabant Cloud ARK Identifier"
                obj_dict['name'] = obj['dedicationURI']['Concept']['prefLabel']
                try:
                    obj_dict['alt_names'] = obj['dedicationURI']['Concept']['altLabel']
                except:
                    pass
                try:
                    obj_dict['other_identifier_content'] = obj['dedicationURI']['Concept']['wikidata']
                    obj_dict['other_identifier_type'] = 'Wikidata'
                except:
                    pass

                obj_list.append(obj_dict)
                obj_dict = {}

            elif isinstance(obj['dedicationURI'], list):   
                
                for dedication in obj['dedicationURI']:                
                
                    obj_dict['ResourceID'] = obj['ResourceID']
                    obj_dict['card'] = "dedications"
                    obj_dict['card_order'] = '6'
                    dedication_uri = str(uuid.uuid5(uuid.NAMESPACE_DNS, dedication['Concept']['id']))
                    resourceXresourceId = str(uuid.uuid5(uuid.NAMESPACE_DNS, dedication['Concept']['id'] + 'relation'))                
                    obj_dict['dedication_activity_actor'] = '[{"resourceId": "%s", "ontologyProperty": "", "resourceXresourceId": "%s", "inverseOntologyProperty": ""}]' % (dedication_uri, resourceXresourceId)
                    obj_dict['dedication_activity_actor_role'] = 'Dedicated to'
                    # for actors
                    obj_dict['resource_id'] = dedication_uri
                    obj_dict['identifier_content'] = obj['id']
                    obj_dict['identifier_type'] = "Brabant Cloud ARK Identifier"

                    obj_dict['name'] = dedication['Concept']['prefLabel']
                    try:
                        obj_dict['alt_names'] = dedication['Concept']['altLabel']
                    except:
                        pass
                    try:
                        obj_dict['other_identifier_content'] = dedication['Concept']['wikidata']
                        obj_dict['other_identifier_type'] = 'Wikidata'
                    except:
                        pass    
                    obj_list.append(obj_dict)
                    obj_dict = {}
            
            #resourceXresourceId = str(uuid.uuid5(uuid.NAMESPACE_DNS, obj_dict['material_location_street_address_city'] + 'relation'))
            #obj_dict['material_location_place'] = 
        
        
    d_source_df = pd.DataFrame(obj_list)    
    number_of_records = len(obj_list)
        
    if number_of_records > 0:                       
        #save_it = save_mapped_set(source_df, package, resource_model, card)
        d_source_df.to_csv(f'out/buildings/dedications.csv', index =False)
        #return '%s saved: %s' % (card_name, number_of_records)
        return d_source_df

## Actors

In [25]:
def make_actors(architects, dedications, card_name, card_order):
    
    alt_names_df = pd.DataFrame
    alt_names_df = pd.DataFrame(columns=['ResourceID', 'name_fullname'])


    actors_df = pd.concat([architects,dedications])

    actors_df['card'] = 'Actors'
    actors_df['card_order'] = '0'
    actors_df['name_type'] = 'Primary'
    actors_df['name_language'] = 'Nederlands'
    
    
    actors_df = actors_df.drop(columns=['ResourceID', 'production_event_actor', 'production_event_actor_role'])

    actors_df.rename(columns={"resource_id": "ResourceID","name": "name_givenname","wikidata": "resource_id","wikidata_type": "other_identifier_type"}, inplace=True)

    actors_df = actors_df.drop_duplicates(subset=['ResourceID'])
    number_of_records = actors_df.shape[0]
    unique_list = actors_df.to_dict('records')
        
    if number_of_records > 0:                       
        #save_it = save_mapped_set(source_df, package, resource_model, card)
        actors_df.to_csv(f'out/actors/actors.csv', index =False)
        #return '%s saved: %s' % (card_name, number_of_records)
        return unique_list

## Alternate names

In [26]:
def make_alternate_names(source_list, node_mappings, graph, dataset, card_name, card_order):
    
    obj_dict = {}
    obj_list = []    
    for obj in source_list:
        if 'alt_names' in obj.keys():            
            if isinstance(obj['alt_names'], dict):  
                obj_dict['ResourceID'] = obj['ResourceID']
                obj_dict['id'] = obj['identifier_content']
                obj_dict['card'] = card_name
                obj_dict['card_order'] = '1'
                obj_dict['alternative_name_content'] = alt_label
                obj_dict['alternative_name_content_type'] = 'Alternative'
                obj_dict['alternative_name_content_language'] = 'Nederlands'
                          
                obj_list.append(obj_dict)
                obj_dict = {}
            
            elif isinstance(obj['alt_names'], list): 
                for alt_name in obj['alt_names']:
                    obj_dict['ResourceID'] = obj['ResourceID']
                    obj_dict['id'] = obj['identifier_content']
                    obj_dict['card'] = card_name
                    obj_dict['card_order'] = '1'
                    obj_dict['alternative_name_content'] = alt_name
                    obj_dict['alternative_name_content_type'] = 'Alternative'
                    obj_dict['alternative_name_content_language'] = 'Nederlands'
                    
                    obj_list.append(obj_dict)
                    obj_dict = {}
        
        
    source_df = pd.DataFrame(obj_list)
    number_of_records = len(obj_list)
        
    if number_of_records > 0:                       
        #save_it = save_mapped_set(source_df, package, resource_model, card)
        source_df.to_csv(f'out/actors/actors_alternate_names.csv', index =False)
        #return '%s saved: %s' % (card_name, number_of_records)
        return obj_list

## Generate dataset

In [27]:
%%time

with open(mappings_path + 'mappings_bg.json') as f:
    mappings_dict = json.load(f)  
    
    record_dict = {}
    record_list = []

    actors_list = []
    dedications_list = []
    architects_list = []

    name = mappings_dict['name']
    graph = mappings_dict['graph']
    print('-', name)
    print('  -', graph)
    
    for cards, cards_values in mappings_dict['mappings'].items():  
        card_name = cards
        for item,item_value in cards_values.items():
            if item == 'dataset': 
                dataset = item_value
            if item == 'card_order':
                card_order = item_value
            if item == 'filter':
                card_filter = item_value    
            if item == 'node_mappings':
                node_mappings = item_value    
           
        node_mappings_keys = [item['from'] for item in node_mappings]
        #records = csv.DictReader(open(f'{source_path}{dataset}', encoding='utf-8-sig'), delimiter=',')
        with open(source_file_name) as f:
            records = json.load(f)

            _landmarks = records['records']['record']

            for _record in _landmarks:                 ## add filter from mapping!  
                record = _record['LandmarksOrHistoricalBuildings']
                record_dict['ResourceID'] = str(uuid.uuid5(uuid.NAMESPACE_DNS, record['id']))
                record_dict['card'] = card_name
                record_dict['card_order'] = card_order

                for mapping_key in node_mappings_keys:
                    if mapping_key in record.keys():
                        if len(record[mapping_key]) > 0:                                
                            record_dict[mapping_key] = record[mapping_key]
                    
                record_list.append(record_dict)                            
                record_dict = {}
            
            if card_name == 'actors':                
                actors_list.extend(record_list)                    
            if card_name == 'dedications':                
                dedications_list.extend(record_list)                    
            if card_name == 'architects':                
                architects_list.extend(record_list)                    


            record_list = []


    if len(architects_list) > 0:
        Architects = make_achitects(architects_list, node_mappings, graph, dataset, card_name, card_order)
        print('    -', "Architects", len(Architects))
    
    if len(dedications_list) > 0:
        Dedications = make_dedications(dedications_list, node_mappings, graph, dataset, card_name, card_order)
        print('    -', "Dedications", len(Dedications))

    if len(actors_list) > 0:
        Actors = make_actors(Architects, Dedications, 'Actors', '4')
        print('    -', "Actors", len(Actors))

    if len(actors_list) > 0:        
        AlternateName = make_alternate_names(Actors, '', graph, dataset, 'Alternate names', card_order)
        print('    -', "AlternateName", len(AlternateName))


print('---------') 
print(datetime.datetime.now())
#print(json.dumps(Dedications, indent=2))
#print('---------------')
#print(json.dumps(AlternateName, indent=2))
#%reset -f 

- brabantse_gebouwen
  - Building
    - Architects 605
    - Dedications 362
    - Actors 371
    - AlternateName 796
---------
2025-06-09 17:55:54.497232
CPU times: user 228 ms, sys: 43.6 ms, total: 271 ms
Wall time: 298 ms


In [ ]:
print(uuid.uuid5(uuid.NAMESPACE_DNS, 'https://n2t.net/ark:/15052/6dc0f577-b706-4f18-83a8-50548f252d6f'))
print(uuid.uuid5(uuid.NAMESPACE_DNS, 'https://n2t.net/ark:/15052/6dc0f577-b706-4f18-83a8-50548f252d6f'))